In [1]:
import spacy
nlp = spacy.load("en_core_web_lg")

import numpy as np
import re
import importlib
import time
from typing import Optional

import ogree_wiki as wiki
importlib.reload(wiki)

<module 'ogree_wiki' from 'c:\\Users\\Admin\\Documents\\IMT Atlantique\\UE -Projet Entreprise-\\OGrEE_NLP\\ogree_wiki.py'>

In [3]:
ACTIONS_DEFAULT = {
                    "ACTION_POSITIVE" : ["make","build","put","place"],
                    "ACTION_NEGATIVE" : ["remove", "delete"], 
                    "ALTERATION" : ["modify", "change","move","set","rename","rotate"]
                    }

ACTIONS_CLI = {
                "ACTION_POSITIVE" : "+",
                "ACTION_NEGATIVE" : "-"
                }

SIMILARITY_PARAMETER = 0.5

KEY_WORDS_DICT = {
            "name" : ["name","called"],
            "position" : ["position", "located"],
            "rotation" : ["rotation","turned","degree"],
            "size" : ["size","dimensions"],
            "template" : ["template"],
            "axisOrientation" : ["orientation", "axisOrientation"], #CAN BE IMPROVED !!!!!!!!!
            "slot" : ["slot"],
            "color" : ["color"], #Est-ce qu'on rajoute des couleurs genre red, green, grey, ...?
            "unit" :  ["unit"],    # C'est quoi le mieux ? ["t","m","f",'metre', "tilde", "feet"],
            "floorUnit" : ["floorUnit"],
            "side" : ["side"], #Can we just put all the side in this list : it could make our code more resilient
            "temperature" : ["temperature"], #Same here with cold or warm
          #  "reserved" : ["reserved"],   
           # "technical" : ["technical"]
            }


In [4]:
def findIndexAction(processed_entry : type(nlp("")), indexAction : int, list_index_entities : list) -> int :
    # identify relations for each entity recognized
    currentIndexes = list_index_entities[::]
    counter = 0
    while counter < 3 :
        for i in range(len(list_index_entities)) :
            currentWord = processed_entry[currentIndexes[i]].head
            currentIndexes[i] = list(processed_entry).index(currentWord)
        counter += 1

    currentIndexesBool = [index == indexAction for index in currentIndexes]
    if sum(currentIndexesBool) >= 2 :
        currentIndexDistance = [(abs(indexAction - list_index_entities[i]),list_index_entities[i]) for i in range(len(currentIndexes)) if currentIndexesBool[i] == True]
        return min(currentIndexDistance)[1]
    elif sum(currentIndexesBool) == 1:
        return list_index_entities[currentIndexesBool.index(True)]
    else :
        raise Exception("Interpretation issue : no relation found, please try again")


In [5]:
def findIndexMainSubject(processed_entry : type(nlp("")), indexAction : int, dictioIndexKeyWords : dict) -> int :

    counter = 0
    currentIndexes = {index:index for index,parameter in dictioIndexKeyWords.items() if parameter not in ACTIONS_DEFAULT.keys()}
    currentWords = {index:processed_entry[index] for index,parameter in dictioIndexKeyWords.items() if parameter not in ACTIONS_DEFAULT.keys()}
    while (not indexAction in currentIndexes.values()) and counter < 3 :
        currentWords = {originIndex : processed_entry[currentIndex].head for originIndex,currentIndex in currentIndexes.items()}
        currentIndexes = {originIndex : list(processed_entry).index(currentWords[originIndex]) for originIndex,_ in currentIndexes.items()}
        counter += 1

    if counter == 3 :
        raise Exception("Did not find the main action")
    
    if list(currentIndexes.values()).count(indexAction) != 1 :
        listIndexesRemaining = [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction and originIndex > indexAction]
        return listIndexesRemaining[0]
    else :
        return [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction][0]


In [6]:
def name(processed_entry : type(nlp("")), dictioEntities : dict, indexesMain : list) -> int :

    def findClose(processed_entry : type(nlp("")), index : int) -> (int|None) :
        if index +1 <= len(processed_entry)-1 :
            if str(processed_entry[index+1]).isupper() or processed_entry[index+1].pos_ == "PROPN" :
                return index+1
        if 0 <= index -1 :
            if str(processed_entry[index-1]).isupper() or processed_entry[index-1].pos_ == "PROPN" :
                return index-1
        return None
    

    def findAttachedEntity(processed_entry : type(nlp("")), index : int) -> (int|None) : 
        counter = 0
        isFound = False
        currentIndex = index
        currentWord = processed_entry[currentIndex]
        while (not isFound) and counter < 3 :
            currentWord = processed_entry[currentIndex].head
            currentIndex = list(processed_entry).index(currentWord)
            if str(currentWord) in dictioEntities.values() :
                return currentIndex
            if currentWord.pos_ == "VERB" and currentIndex == indexesMain[1] : 
                return indexesMain[0]
            counter += 1
        return None
    
    EXPLICIT =  KEY_WORDS_DICT["name"]
    IMPLICIT = ["current","main"]

    dictioEntityNames = {}
        
    for index,entity in dictioEntities.items() :
    # if the name if right beside the entity
        resultIndex = findClose(processed_entry, index)
        if resultIndex != None :
            dictioEntityNames[index] = resultIndex

    if len(dictioEntityNames) < len(dictioEntities) : # if not all names found

        for index,token in enumerate(processed_entry) : # look for keyword

            if str(processed_entry[index]).isupper() or processed_entry[index].pos_ == "PROPN" :
                indexAttachedEntity = findAttachedEntity(processed_entry,index)
                if not (indexAttachedEntity in dictioEntityNames or indexAttachedEntity == None) :
                    dictioEntityNames[indexAttachedEntity] = index
            
            # if the token is a synonym of called
            if sum([token.similarity(nlp(word)[0]) > SIMILARITY_PARAMETER for word in EXPLICIT]) >= 1 :
                resultIndex = findClose(processed_entry, index)
                if resultIndex != None :
                    indexAttachedEntity = findAttachedEntity(processed_entry,index)
                    if not (indexAttachedEntity in dictioEntityNames or indexAttachedEntity == None) :
                        dictioEntityNames[indexAttachedEntity] = resultIndex


    if len(dictioEntityNames) < len(dictioEntities) : # if still not all names found
        pass
    # TODO : get current names

    # TODO : check for implicit words now
    
    return dictioEntityNames

In [7]:
def position(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity = "") -> Optional[list] :
    # search for keywords to the right and left to split whole sentence into parts where to look for param entries
    
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    position_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in next_words]))
    if not (len(position_list) == 2 or (len(position_list) == 3 and entity == "rack") or (len(position_list) ==1 and entity =="device")):
        position_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in last_words]))
        if not (len(position_list) == 2 or (len(position_list) == 3 and entity == "rack")) :
            raise Exception("Wrong location format entered")
        else :
            position_list = position_list_left
    result = [float(coord) for coord in position_list] if position_list else None
    return result

In [8]:
text = nlp("build a device named RACK1 at the postion =10")
print(position(text,7,0,len(text),"device"))

[10.0]


In [9]:
def axisOrientation(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    # We search the axis Orientation
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    #An axis Orientation  can be any combinason of [+/-]x[+/-]y. eg: +x+y or -x+y
    print(" ".join([str(token) for token in next_words]))
    axisOrientationList =  re.findall(" (-x|\+[ *]x)[ *](-y|\+[ *]y)", " ".join([str(token) for token in next_words]))
    #If we didn't find an axis orientation, we need to seek it in the second part of the sentence
    if len(axisOrientationList) ==0 :
        axisOrientationList = re.findall("(-x|\+[ *]x)[ *](-y|\+[ *]y)", " ".join([str(token) for token in last_words]))
        #If we still didn't find it, we need to raise an error
        if len(axisOrientationList) ==0 :
            raise Exception("Wrong axisOrientation format entered")
    
    #We need to raise an error if we find only one coordonate
    if len(axisOrientationList[0]) < 2:
        raise Exception("Only one coordinate was found")
    #Even if several axis Orientations where found, we always take the first one
    return axisOrientationList[0][0].replace(" ", "") + axisOrientationList[0][1].replace(" ", "")


In [10]:
def unit(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    #This function help us to find the unit of the floor
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    #An unit can be m, t , f, meters, tildes, feets
    unitList = re.findall(r'\bt\b|\bf\b|\bm\b|\bu\b'," ".join([str(token) for token in next_words]))
    unitList += re.findall("metre|feet|tilde|unit"," ".join([str(token) for token in next_words]))
    #If we didn't find any unit, we need to seek it in the second part of the sentence
    if len(unitList) ==0:
        unitList = re.findall(r'\bt\b|\bf\b|\bm\b|\bu\b'," ".join([str(token) for token in last_words]))
        unitList += re.findall("metre|feet|tilde|unit"," ".join([str(token) for token in last_words]))
        
        #We should have found an unit
        if len(unitList) == 0:
            raise Exception("There wasn't any unit in the sentence")
    
    #If there are many unit, we choose the first one
    
    return unitList[0]

In [16]:
def color(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    #This function seek the hexadecimal color next to the word color (or one of its synonym)
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    print("next_words" ,next_words)
    print("last_words" ,last_words)
    #A color should be in hexadecimal ==> MAYBE A NAME LATER ??
    colorList = re.findall(r'#[ *][A-Z0-9a-z]{6}\b'," ".join([str(token) for token in next_words]))

    #If we didn't find a color after the word color, we need to look before
    if len(colorList) ==0:
        colorList = re.findall(r'#[ *][A-Z0-9a-z]{6}\b'," ".join([str(token) for token in last_words]))

        #If there isn't any color, we need to raise an execption
        if len(colorList) == 0: 
            raise Exception("There wasn't any color")
        
    #If there are several colors, we choose the first one
    return colorList[0].replace(" ", "")

In [11]:
def temperature(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    #This function helps to find if its cold or warm
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    
    #A temperature is either cold or warm 
    tempList = re.findall("cold|warm", " ".join([str(token) for token in next_words]))
    
    #If we didn't find it after the word temperature, we need to seek before
    if len(tempList) ==0:
        tempList = re.findall("cold|warm", " ".join([str(token) for token in last_words]))

        #If neither cold or warm was found we need to raise an Exception
        if len(tempList) == 0:
            raise Exception("Neither cold or warm was found in the sentence")

    #If there are several temperature, we choose the first one
    return tempList[0]
    


In [12]:
def side(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]

    #A side is either front, rear, frontflipped,rearflipped 
    sideList = re.findall("front | rear | frontflipped | rearflipped", " ".join([str(token) for token in next_words]))
    #if a side was not found after the word, we need to seek after
    if len(sideList) == 0:
        sideList = re.findall("front | rear | frontflipped | rearflipped", " ".join([str(token) for token in last_words]))

        #If the side wasn't after the word side, there is an error
        if len(sideList) == 0: 
            raise Exception( "there wasn't any side in the sentence")

    #If there are several sides, we take the first one
    return sideList[0].replace(" ","")


In [13]:
def floorUnit(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    un =unit(processed_entry, index, lastKeyWordIndex, nextKeyWordIndex)
    if un.replace(" ", "") == "u":
        raise Exception ("A floor Unit can't have u as unit")
    return un 

In [14]:
def slot(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    # TO DO : ask THEO to use the function name here
    pass
    

In [17]:
testNlp = nlp("create a room called ROOM1 with a  temperature warm with an orientation of -x -y  in u with #abD874 #ABDDDD as color with rear as side   ")

print("axisOrientation : ", axisOrientation(testNlp, 0,0,len(testNlp) -1))
print("Unit : ", unit(testNlp,0,0,len(testNlp) -1))
print("Color : ", color(testNlp,22,0,len(testNlp) -1))
print("Temperature", temperature(testNlp,12,0,len(testNlp) -1))
print("Side : ", side(testNlp, 20,0,len(testNlp) -1) )

a room called ROOM1 with a   temperature warm with an orientation of -x -y   in u with # abD874 # ABDDDD as color with rear as side
axisOrientation :  -x-y
Unit :  u
next_words ABDDDD as color with rear as side
last_words create a room called ROOM1 with a  temperature warm with an orientation of -x -y  in u with #abD874
Color :  #abD874
Temperature warm
Side :  rear


In [18]:
def rotation(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int, entity="") -> Optional[float] :
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    
    isRotationNegative = False
    print("next_word : "," ".join([str(token) for token in next_words]))
    rotation_list = re.findall("([-]*[0-9]+[.]*[0-9]*){1,3}", " ".join([str(token) for token in next_words]))
    print("rotation_list : ", rotation_list)
    isRotationNegative = re.search("counter.*clockwise", "".join([str(token) for token in next_words]))
    if not len(rotation_list) == 1 and not len(rotation_list) == 3 :
        rotation_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in last_words]))
        isRotationNegative = re.search("counter.*clockwise", "".join([str(token) for token in next_words]))
        if not len(rotation_list_left) == 1 :
            raise Exception("Wrong rotation format entered")
        else :
            rotation_list = rotation_list_left
    rotationFinal = rotation_list[0] if len(rotation_list) == 1 else rotation_list
    rotationFinal = -rotationFinal if isRotationNegative else rotationFinal
    return rotationFinal

In [19]:
text = nlp("create a corridor with a rotation of 90 90 90")
print(rotation(text,4,0,len(text)))

next_word :  rotation of 90 90 90
rotation_list :  ['90', '90', '90']
['90', '90', '90']


In [20]:
def size(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int, entity="") -> Optional[list] :
    #it only works for regular size not sizeXY or sizeU
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]

    size_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in next_words]))
    print(size_list)
    if not (len(size_list) == 3) and not( entity =="pillar" and len(size_list) == 2 ) and not(len(size_list)==1 and entity == "device"):
        size_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in last_words]))
        if not (len(size_list) == 3) and not( entity =="pillar" and len(size_list) == 2 ) and not(len(size_list)==1 and entity == "device"):
            raise Exception("Wrong location format entered")
        else :
            size_list = size_list_left
    result = [float(coord) for coord in size_list] if size_list else None
    return result

In [21]:
text = nlp("create the pillar with 0 0  as size")
print("Size  : ", size(text, 0,0,len(text)-1,"pillar"))

['0', '0']
Size  :  [0.0, 0.0]


In [22]:
def template(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int, entity : str) -> Optional[str] :
    #The template is the defined by a name, it's the same structure as for Name() but simplified since there is less convuloted wording
    
    def findClose(indexToken : int) -> Optional[int] :
        if indexToken +1 <= len(processed_entry)-1 :
            if str(processed_entry[indexToken+1]).isupper() or processed_entry[indexToken+1].pos_ == "PROPN" :
                return indexToken+1
        if 0 <= indexToken -1 :
            if str(processed_entry[indexToken-1]).isupper() or processed_entry[indexToken-1].pos_ == "PROPN" :
                return indexToken-1
        return None

    def findAttachedEntity(indexToken : int) -> (bool) : 
        counter = 0
        isFound = False
        currentIndex = indexToken
        currentWord = processed_entry[currentIndex]
        while (not isFound) and counter < 3 :
            currentWord = processed_entry[currentIndex].head
            currentIndex = list(processed_entry).index(currentWord)
            if str(currentWord) in KEY_WORDS_DICT["template"] : #Peux être faut il passer aux Lemma ?
                return True
            counter += 1
        return False
    
    EXPLICIT =  KEY_WORDS_DICT["name"]

    # if the name if right beside the entity
    resultIndex = findClose(index)
    if resultIndex != None :
        return processed_entry[resultIndex]
    else: #look in the NProp if one is linked to Template
        for indexTemp,tokenTemp in enumerate(processed_entry) : # look for keyword
            if str(processed_entry[indexTemp]).isupper() or processed_entry[indexTemp].pos_ == "PROPN" :
                indexAttachedEntity = findAttachedEntity(indexTemp)
                if findAttachedEntity(indexTemp) :
                    return tokenTemp
            
            """
            # if the token is a synonym of called
            if sum([tokenTemp.similarity(nlp(word)[0]) > SIMILARITY_PARAMETER for word in EXPLICIT]) >= 1 :
                resultIndex = findClose(indexTemp)
                if resultIndex != None :
                    indexAttachedEntity = findAttachedEntity(indexTemp)
                    if indexAttachedEntity != None :
                        resultIndex = indexAttachedEntity
                else {
                    return resultIndex
                }
            """

    return None

In [24]:
pip install textblob

     ---------------------------------------- 0.0/636.8 kB ? eta -:--:--
     ----- --------------------------------- 92.2/636.8 kB 1.8 MB/s eta 0:00:01
     --------- ---------------------------- 163.8/636.8 kB 2.0 MB/s eta 0:00:01
     ---------------- --------------------- 276.5/636.8 kB 2.1 MB/s eta 0:00:01
     ------------------------- ------------ 419.8/636.8 kB 2.0 MB/s eta 0:00:01
     ----------------------------- -------- 491.5/636.8 kB 2.2 MB/s eta 0:00:01
     -------------------------------------  634.9/636.8 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 636.8/636.8 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---- ----------------------------------- 0.2/1.5 MB 3.5 MB/s eta 0:00:01
     ------- -------------------------------- 0.3/1.5 MB 3.5 MB/s eta 0:00:01
     --------- ------------------------------ 0.4/1.5 MB 2.9 MB/s eta 0:00:01
     ------------- -------------------------- 0.5/1.5 MB 3.0 M


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [36]:
from textblob import TextBlob
 
a = "cmputr"           # incorrect spelling
print("original text: "+str(a))
 
b = TextBlob(a)
 
# prints the corrected spelling
print("corrected text: "+str(b.correct()))

test = nlp("crete a site calld SITE 0 0 0")

for word in test:
    print("Old word : " , word)
    b = TextBlob(str(word))
    print("corrected text: "+str(b.correct()))

original text: cmputr
corrected text: computer
Old word :  crete
corrected text: create
Old word :  a
corrected text: a
Old word :  site
corrected text: site
Old word :  calld
corrected text: called
Old word :  SITE
corrected text: SITE
Old word :  0
corrected text: 0
Old word :  0
corrected text: 0
Old word :  0
corrected text: 0


In [390]:
def main() -> str :
    FINAL_INSTRUCTION = ""

    ENTITIES_FULL_NAME = {"entity" : list(wiki.ENTITIES.keys())}

    # TODO : add already existing entity names
    KEY_WORDS_ALL = {**ENTITIES_FULL_NAME,  **KEY_WORDS_DICT}
    # for parameter in KEY_WORDS_ALL.keys() :
    #     for word in KEY_WORDS_ALL[parameter] :
    #         print(word,nlp(word)[0].pos_)
    KEY_WORDS_ENTRY = {}

    natural_entry = input("Enter a prompt. Please follow the instructions given.\n")
    processed_entry = nlp(natural_entry)

    # we detect key words in the sentence given and put them into KEY_WORDS_ENTRY
    for index,token in enumerate(processed_entry) :
        matching_list = []
        if token.pos_ == "VERB" and str(token) == token.lemma_ : # infinitive verb
            for parameter in ACTIONS_DEFAULT.keys() :
                similarity = max([token.similarity(nlp(word)[0]) for word in ACTIONS_DEFAULT[parameter]])
                matching_list.append((similarity,parameter))
        elif token.pos_ in ["NOUN","ADP","VERB"]:
            for parameter in KEY_WORDS_ALL.keys() : 
                similarity = max([token.similarity(nlp(word)[0]) for word in KEY_WORDS_ALL[parameter]])
                matching_list.append((similarity,parameter))
        else :
            continue
        match = max(matching_list)
        if match[0] > SIMILARITY_PARAMETER :
            KEY_WORDS_ENTRY[index] = match[1]

    print(KEY_WORDS_ENTRY)

    # test detection
    list_key_param = list(KEY_WORDS_ENTRY.values())
    count_action = 0
    for action_type in ACTIONS_DEFAULT.keys() :
        count_action += list_key_param.count(action_type)

    if count_action != 1 :
        raise Exception("Action not detected")
    else :
        global INDEX_ACTION
        global INDEX_MAIN_SUBJECT
        INDEX_ACTION = [index for index,keyword in KEY_WORDS_ENTRY.items() if keyword in ACTIONS_DEFAULT.keys()][0]
        INDEX_MAIN_SUBJECT = findIndexMainSubject(processed_entry, INDEX_ACTION, KEY_WORDS_ENTRY)  
    print("index main : ",INDEX_MAIN_SUBJECT)

    dictEntities = {index : str(processed_entry[index]) for index,keyword in KEY_WORDS_ENTRY.items() if keyword == "entity"}
    print(dictEntities)
    dictioEntityNames = name(processed_entry, dictEntities, [INDEX_MAIN_SUBJECT, INDEX_ACTION])
    print("names : ",dictioEntityNames)

    # TODO : match cases (main = entity/parameter, verb = +/alteration...)
    print("KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] == entity :", KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] == "entity")
    if KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] == "entity" :
          # we do the processes related to each parameter
        dictioEntityParameters = wiki.makeDictParam(str(processed_entry[INDEX_MAIN_SUBJECT]))
        dictioEntityParameters["name"] = str(processed_entry[dictioEntityNames[INDEX_MAIN_SUBJECT]])
        print("dictioEntityParameters : ", dictioEntityParameters)
        allEntryItems = KEY_WORDS_ENTRY.items()
        print("allEntryItems : ", allEntryItems)
        for counter,(index,parameter) in enumerate(allEntryItems) :
            if (parameter in KEY_WORDS_DICT.keys()) and (parameter in dictioEntityParameters.keys()) and parameter != "name":
                lastKeyWordIndex = 0 if counter == 0 else list(allEntryItems)[counter-1][0]
                nextKeyWordIndex = len(processed_entry) if counter == len(allEntryItems)-1 else list(allEntryItems)[counter+1][0]
                # get the parameter value
                print("parameter : ", parameter)
                parameterValue = globals()[parameter](processed_entry, index, lastKeyWordIndex, nextKeyWordIndex,dictEntities[INDEX_MAIN_SUBJECT])
                dictioEntityParameters[parameter] = parameterValue # store the value
        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ACTION_POSITIVE" : 
          

            print(dictioEntityParameters)
            # TODO : call the class method

        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ACTION_NEGATIVE" :
            print(dictioEntityParameters)
           # TODO : call the class method

        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ALTERATION" :
            print(dictioEntityParameters)
            # TODO : understand what are the changes and its consequences
            pass

    if KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] in KEY_WORDS_DICT.keys() :
        # TODO : seek the value to be changed
        pass


    
    # if seeking the name for the main entity, pass the indexaction as parameter
    # if no name found, check the type of action : if +, a name is needed, otherwise not necessarily

    # check if parameters were not given

    # return KEY_WORDS_ENTRY

In [413]:
text0 = "create a site called SITE"
text = "create the building called IMT at the position 0 0 rotated 90 and a size of 10*10*10 "
test2 = """create a room called ROOM1 with a position of 4 14 a rotation of 10
an orientation of -x -y  a size of 10 10 10 with a floorUnit in m """
test3 = """build a rack named RACK1 at the position 10 -10 50 in unit tildes with a rotation of [0,0,180] and a size of 10 20 30"""
test4 = """ construct a device named DEVICE1 at the position 4 at the size 45"""
test5 = """create a corridor called CORR1 at the position 0 0 with metre as unit a rotation of 0 0 0 a size of 10*10*10 and a warm temperature"""
test6 = """create a tag named TAG1 in #154987"""
main()

C:\Users\Admin\AppData\Local\Temp\ipykernel_12732\1551311445.py:25: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity = max([token.similarity(nlp(word)[0]) for word in KEY_WORDS_ALL[parameter]])


{0: 'ACTION_POSITIVE', 2: 'entity', 3: 'name', 7: 'position', 13: 'unit', 15: 'rotation', 21: 'size', 31: 'temperature'}
index main :  2
{2: 'corridor'}


C:\Users\Admin\AppData\Local\Temp\ipykernel_12732\3433111061.py:49: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  if sum([token.similarity(nlp(word)[0]) > SIMILARITY_PARAMETER for word in EXPLICIT]) >= 1 :


names :  {2: 4}
KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] == entity : True
dictioEntityParameters :  {'name': 'CORR1', 'position': None, 'unit': None, 'rotation': None, 'size': None, 'temperature': None}
allEntryItems :  dict_items([(0, 'ACTION_POSITIVE'), (2, 'entity'), (3, 'name'), (7, 'position'), (13, 'unit'), (15, 'rotation'), (21, 'size'), (31, 'temperature')])
parameter :  position
parameter :  unit
parameter :  rotation
next_word :  of 0 0 0 a
rotation_list :  ['0', '0', '0']
parameter :  size
['10', '10', '10']
parameter :  temperature
{'name': 'CORR1', 'position': [0.0, 0.0], 'unit': 'metre', 'rotation': ['0', '0', '0'], 'size': [10.0, 10.0, 10.0], 'temperature': 'warm'}


In [ ]:
def createSiteDict(dictSite : dict) -> dict:
    return dictSite
    

In [368]:
def createBuildingDict(dictBuild : dict) -> dict : #NB COLOR + EXCEPTION FOR FUNCTION
    finalDict = {}
    keys = dictBuild.keys()
    print(keys)
    print("name" in keys)
    if "name" in keys and "position" in keys and "rotation" in keys:
        finalDict["name"] = dictBuild["name"]
        finalDict["position"] = dictBuild["position"]
        finalDict["rotation"] = dictBuild["rotation"]
    else:
        raise Exception("Mandatory argument missing")
    if "size" in keys and "template" in keys:
        if dictBuild["size"] !=None and dictBuild["template"] !=None:
            raise Exception("A size and a template were given")
        elif dictBuild["size"] !=None:
            finalDict["size"] = dictBuild["size"]
        else:
            finalDict["template"] = dictBuild["template"]
    else:
        raise Exception("Neither a size neither a template were given")
    return finalDict

In [371]:
text = {'name': 'IMT', 'position': [0.0, 0.0], 'rotation': '90', 'size': None, 'template': "temp"}

print(createBuildingDict(text))

dict_keys(['name', 'position', 'rotation', 'size', 'template'])
True
{'name': 'IMT', 'position': [0.0, 0.0], 'rotation': '90', 'template': 'temp'}


In [396]:
def createRoomDict(dictRoom : dict) -> dict:
    finalDict = {}
    keys = dictRoom.keys()
    if "name" in keys and "position" in keys and "rotation" in keys:
        finalDict["name"] = dictRoom["name"]
        finalDict["position"] = dictRoom["position"]
        finalDict["rotation"] = dictRoom["rotation"]
    else:
        raise Exception("At least one mandatory element is missing")
    
    if "size" in keys and "template" in keys:
        if dictRoom["size"] !=None and dictRoom["template"] !=None:
            raise Exception("A size and a template were given")
        elif dictRoom["size"] !=None:
            #If we have a size, we have to have an axisOrientation
            if "axisOrientation" in keys and dictRoom["axisOrientation"] !=None:
                finalDict["size"] = dictRoom["size"]
                finalDict["axisOrientation"] = dictRoom["axisOrientation"]
            else :
                raise Exception("axisOrientation missing while a size was given")
            if "floorUnit" in keys and dictRoom["floorUnit"] !=None:
                finalDict["floorUnit"] = dictRoom["floorUnit"]
        else:
            finalDict["template"] = dictRoom["template"]
    else:
        raise Exception("Neither a size neither a template were given")
    return finalDict


In [399]:
test = {'name': 'ROOM1', 'position': [4.0, 14.0], 'rotation': '10', 'size': [10.0, 10.0, 10.0], 'axisOrientation': '-x-y', 'floorUnit': 'm', 'template': None}
print(createRoomDict(test))

{'name': 'ROOM1', 'position': [4.0, 14.0], 'rotation': '10', 'size': [10.0, 10.0, 10.0], 'axisOrientation': '-x-y', 'floorUnit': 'm'}


In [405]:
def createRackDict(dictRack : dict) -> dict:
    finalDict = {}
    keys = dictRack.keys()
    if "name" in keys and "position" in keys and "rotation" in keys and 'unit' in keys:
        finalDict["name"] = dictRack["name"]
        finalDict["position"] = dictRack["position"]
        finalDict["rotation"] = dictRack["rotation"]
        finalDict["unit"] = dictRack["unit"]
    else:
        raise Exception("At least one mandatory element is missing")
    
    if "size" in keys and "template" in keys:
        if dictRack["size"] !=None and dictRack["template"] !=None:
            raise Exception("A size and a template were given")
        elif dictRack["size"] !=None:
            finalDict["size"] = dictRack["size"]
        else:
            finalDict["template"] = dictRack["template"]
    else:
        raise Exception("Neither a size neither a template were given")
    return finalDict

In [406]:
test = {'name': 'RACK1', 'position': [10.0, -10.0, 50.0], 'unit': 'tilde', 'rotation': ['0', '0', '180'], 'size': [10.0, 20.0, 30.0], 'template': None}
print(createRackDict(test))

{'name': 'RACK1', 'position': [10.0, -10.0, 50.0], 'rotation': ['0', '0', '180'], 'unit': 'tilde', 'size': [10.0, 20.0, 30.0]}


In [410]:
def createDeviceDict(dictDevice : dict) -> dict:
    finalDict = {}
    keys = dictDevice.keys()
    if "name" in keys:
        finalDict["name"] = dictDevice["name"]
    else:
        raise Exception("At least one mandatory element is missing")
    #A devive has either a position either a slot
    if "position" in keys and "slot" in keys:
        if dictDevice["position"] !=None and dictDevice["slot"] !=None:
            raise Exception("a posU and a slot were given")
        elif dictDevice["position"] !=None:
            finalDict["position"] = dictDevice["position"]
        else:
            finalDict["slot"] = dictDevice["slot"]
    #A device has either a size either a template
    if "size" in keys and "template" in keys:
        if dictDevice["size"] !=None and dictDevice["template"] !=None:
            raise Exception("A size and a template were given")
        elif dictDevice["size"] !=None:
            finalDict["size"] = dictDevice["size"]
        else:
            finalDict["template"] = dictDevice["template"]
    else:
        raise Exception("Neither a size neither a template were given")
    
    #A device can have a side if it has a template and a slot
    if dictDevice["template"] !=None and dictDevice["slot"] !=None:
        if side in keys and dictDevice["side"] !=None:
            finalDict["side"] = dictDevice["side"]

    return finalDict

In [411]:
test = {'name': 'DEVICE1', 'position': [4.0], 'size': [45.0], 'template': None, 'slot': None, 'side': None}
print(createDeviceDict(test))

{'name': 'DEVICE1', 'position': [4.0], 'size': [45.0]}


In [412]:
def createCorridorDict(dictCorridor : dict) -> dict:
    finalDict = {}
    keys = dictCorridor.keys()
    if "name" in keys and "position" in keys and "rotation" in keys and "unit" in keys and "size" in keys and "temperature" in keys:
        finalDict["name"] = dictCorridor["name"]
        finalDict["position"] = dictCorridor["position"]
        finalDict["rotation"] = dictCorridor["rotation"]
        finalDict["unit"] = dictCorridor["unit"]
        finalDict["size"] = dictCorridor["size"]
        finalDict["temperature"] = dictCorridor["temperature"]
    else:
        raise Exception("At least one mandatory argument was forgotten")
    return finalDict

In [414]:
test = {'name': 'CORR1', 'position': [0.0, 0.0], 'unit': 'metre', 'rotation': ['0', '0', '0'], 'size': [10.0, 10.0, 10.0], 'temperature': 'warm'}
print(createCorridorDict(test))

{'name': 'CORR1', 'position': [0.0, 0.0], 'rotation': ['0', '0', '0'], 'unit': 'metre', 'size': [10.0, 10.0, 10.0], 'temperature': 'warm'}


In [13]:
nlp("rotated")[0].similarity(nlp("rotation")[0])
# [nlp("current")[0].similarity(nlp(word)[0]) for word in ["axis", "orientation"]]

0.6624487638473511

In [14]:
#Test position, size, rotation, template
testNlp = nlp("at 0.0 0 create a building named BATIMENT with the dimension 50 40 60, turned by 90 degrees and with the template called FAKER")

print("Position :" + str(position(testNlp, 0, 0, 4, "building")))
print("Rotation :" + str(rotation(testNlp, 15, 11, 19, "building")))
print("Dimension :" + str(size(testNlp, 10, 6, 14, "building")))
print("Template :" + str(template(testNlp, 22, 19, 23, "building")))

TypeError: position() takes 4 positional arguments but 5 were given

In [63]:
spacy.displacy.render(testNlp, style="dep")